In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import requests #to request to the web site from where you want to do scrapping
import bs4 #for beautiful soup
import pandas as pd
import numpy as np


# In[2]:

delete_it = 26
for rtx in range(50):
    name_number = rtx + 1 + delete_it
    name_number = str(name_number) + '_'
    linkkk = input()
    
    res = requests.get(linkkk)
    soup = bs4.BeautifulSoup (res.text, 'lxml')
    title = soup.select('title')
    h1 = soup.select('h1')
    c_name = h1[0].getText()
    c_name = c_name.replace(' ','_')
    li = soup.select('li')
    b = soup.select('b')
    
    


    # In[3]:


    df_main_info = pd.DataFrame(columns=['value'])
    for i in range(10):
        temp = b[i].getText()
        temp = temp.replace(',','')
        if temp == '':
            temp = float(np.nan)
        else:
            temp = float(temp)
            
        df_main_info.loc[len(df_main_info)] = temp

    df_main_info = df_main_info.rename(index={0: 'Market Cap (cr)', 1:'Current Price (rs)', 2:'52 weeks High (rs)', 3:'52 weeks Low (rs)', 4:'Book Value (rs)', 5:'Stock P/E', 6:'Dividend Yield (%)', 7:'ROCE (%)', 8:'ROE (%)', 9:'Sales Growth (3Yrs) (%)'})


    # In[4]:


    temp=soup.select('a')


    # In[5]:


    #find all html source of tables
    temp=soup.find_all('th') #try td also

    #to make list of tables in a html
    all_headers = []

    for i,t in enumerate(temp):
        all_headers.append(t.getText())


    # In[6]:


    #removing unwanted words
    all_headers_new = []

    for t in all_headers:
        if t=='Compounded Sales Growth' or t=='Compounded Profit Growth' or t=='Stock Price CAGR' or t=='Return on Equity':
            continue
        else:
            all_headers_new.append(t)


    # In[7]:


    Quarters =        []
    Profit_and_Loss = []
    Balance_Sheet =   []
    Cash_Flow =       []
    Ratios =          []

    count = 0
    for h in all_headers_new:
        if h=='':
            count = count + 1
        else:
            if(count==1):
                Quarters.append(h)
            if(count==2):
                Profit_and_Loss.append(h)
            if(count==3):
                Balance_Sheet.append(h)
            if(count==4):
                Cash_Flow.append(h)
            if(count==5):
                Ratios.append(h)


    # In[8]:


    #create 5 empty data frames for 5 tables (and adding column name)
    #adding rows and naming of rows are still left
    df_Quarters = pd.DataFrame(columns=Quarters)
    df_Profit_and_Loss = pd.DataFrame(columns=Profit_and_Loss)
    df_Balance_Sheet = pd.DataFrame(columns=Balance_Sheet)
    df_Cash_Flow = pd.DataFrame(columns=Cash_Flow)
    df_Ratios = pd.DataFrame(columns=Ratios)


    # In[9]:


    #find all html source of tables
    temp=soup.find_all('td') #try td also

    #to make list of tables in a html
    all_rows = []
    additional_data_index = 0
    flag = 0

    for i,t in enumerate(temp):
        all_rows.append(t.getText())
        if(t.getText() == "10 Years:" and flag == 0):
            additional_data_index = i
            flag = 1


    # In[10]:


    ############### additional data ######################
    df_Compounded_Sales_Growth = pd.DataFrame(columns=['10 Years','5 Years','3 Years','TTM'])
    df_Compounded_Profit_Growth = pd.DataFrame(columns=['10 Years','5 Years','3 Years','TTM'])
    df_Stock_Price_CAGR = pd.DataFrame(columns=['10 Years','5 Years','3 Years','1 Year'])
    df_Return_on_Equity = pd.DataFrame(columns=['10 Years','5 Years','3 Years','Last Year'])


    # In[11]:


    temp_row1 = [all_rows[additional_data_index+1],all_rows[additional_data_index+3],all_rows[additional_data_index+5],all_rows[additional_data_index+7]]
    temp_row2 = [all_rows[additional_data_index+9],all_rows[additional_data_index+11],all_rows[additional_data_index+13],all_rows[additional_data_index+15]]
    temp_row3 = [all_rows[additional_data_index+17],all_rows[additional_data_index+19],all_rows[additional_data_index+21],all_rows[additional_data_index+23]]
    temp_row4 = [all_rows[additional_data_index+25],all_rows[additional_data_index+27],all_rows[additional_data_index+29],all_rows[additional_data_index+31]]


    # In[12]:


    temp_row_new = []

    for t in temp_row1:
        temp = t
        if temp[0] == '%':
            temp_row_new.append(float(np.nan))
        elif temp == "None%":
            temp_row_new.append(float(0))
        else:
            temp = temp.replace(',','')
            temp = temp.replace('%','')
            temp_row_new.append(float(temp))

    df_Compounded_Sales_Growth.loc[0] = temp_row_new
    df_Compounded_Sales_Growth = df_Compounded_Sales_Growth.rename(index={0: 'value %'})

    #..................................................................

    temp_row_new = []

    for t in temp_row2:
        temp = t
        if temp[0] == '%':
            temp_row_new.append(float(np.nan))
        elif temp == "None%":
            temp_row_new.append(float(0))
        else:
            temp = temp.replace(',','')
            temp = temp.replace('%','')
            temp_row_new.append(float(temp))

    df_Compounded_Profit_Growth.loc[0] = temp_row_new
    df_Compounded_Profit_Growth = df_Compounded_Profit_Growth.rename(index={0: 'value %'})

    #........................................................................

    temp_row_new = []

    for t in temp_row3:
        temp = t
        if temp[0] == '%':
            temp_row_new.append(float(np.nan))
        elif temp == "None%":
            temp_row_new.append(float(np.nan))
        else:
            temp = temp.replace(',','')
            temp = temp.replace('%','')
            temp_row_new.append(float(temp))

    df_Stock_Price_CAGR.loc[0] = temp_row_new
    df_Stock_Price_CAGR = df_Stock_Price_CAGR.rename(index={0: 'value %'})

    # .....................................................................

    temp_row_new = []

    for t in temp_row4:
        temp = t
        if temp[0] == '%':
            temp_row_new.append(float(np.nan))
        elif temp == "None%":
            temp_row_new.append(float(0))
        else:
            temp = temp.replace(',','')
            temp = temp.replace('%','')
            temp_row_new.append(float(temp))

    df_Return_on_Equity.loc[0] = temp_row_new
    df_Return_on_Equity = df_Return_on_Equity.rename(index={0: 'value %'})


    # In[13]:


    #removing unwanted spaces, tbas,'Notes' etc
    all_rows_new = []
    remove_next_flag = False

    for i in range(len(all_rows)):
        temp = all_rows[i]
        temp = temp.replace('\n','')
        temp = temp.replace('  ','')
        temp = temp.replace('\xa0',' ')

        if(not(temp == "")):
            if(remove_next_flag == True):
                remove_next_flag = False
            elif(temp[-1] == ":"):
                remove_next_flag = True
                continue
            elif(temp == 'Notes'):
                all_rows_new.append('Notes')
            else:
                all_rows_new.append(temp)
        else:
            all_rows_new.append("empty")


    # In[14]:


    row_names = []

    for r in all_rows_new:
        if r[0].isdigit() == False and (not (r[0] == "-") and (not r == 'empty')): #if row name comes
            row_names.append(r)


    # In[15]:


    #to measure no. of rows in a table
    rows_in_each_table = {'Quarters':11, 'Profit & Loss':12, 'Balance Sheet':10, 'Cash Flow':4, 'Ratios':3 }


    # In[16]:


    #to measure no. of columns in a table
    columns_in_each_table = {'Quarters':len(df_Quarters.columns), 'Profit & Loss':len(df_Profit_and_Loss.columns), 'Balance Sheet':len(df_Balance_Sheet.columns), 'Cash Flow':len(df_Cash_Flow.columns), 'Ratios':len(df_Ratios.columns)}


    # In[17]:


    #splitting all_rows_new into 5 parts (coz 5 tables)
    table1 = []
    table2 = []
    table3 = []
    table4 = []
    table5 = []

    counter = 0
    for r in all_rows_new:
        if r == 'Sales +' or r == 'Share Capital +' or r == 'Cash from Operating Activity +' or r == 'ROCE %':
            counter = counter + 1
        
        if counter == 1:
            table1.append(r)
        if counter == 2:
            table2.append(r)
        if counter == 3:
            table3.append(r)
        if counter == 4:
            table4.append(r)
        if counter == 5:
            table5.append(r)
            
    #dropping 12 Notes and 1 blank index name
    rangee = columns_in_each_table['Quarters']
    for i in range(rangee+1):
        table1.pop()


    # In[18]:


    table1_data_only = []
    for t in table1:
        if t[0].isdigit() == False and (not (t[0] == "-") and (not (t == "empty"))):
            continue
        else:
            table1_data_only.append(t)


    # In[19]:


    count = 0
    for i in range(rows_in_each_table['Quarters']):
        temp_row = []
        for j in range(columns_in_each_table['Quarters']):
            temp_element = table1_data_only[count]
            temp_element = temp_element.replace(',','')
            temp_element = temp_element.replace('%','')
            
            if temp_element == 'empty':
                temp_element = np.nan
            
            temp_row.append(float(temp_element))
            count = count + 1
        df_Quarters.loc[len(df_Quarters)] = temp_row


    # In[20]:


    #now renaming index for table1
    df_Quarters = df_Quarters.rename(index={0: 'Sales', 1: 'Expenses',2: 'Operating Profit', 3: 'OPM %',4: 'Other Income', 5: 'Interest',6: 'Depreciation', 7: 'Profit before tax',8: 'Tax %', 9: 'Net Profit', 10: 'EPS in Rs'})


    # In[21]:


    table2_data_only = []
    for t in table2:
        if t[0].isdigit() == False and (not (t[0] == "-") and (not (t == "empty"))):
            continue
        else:
            table2_data_only.append(t)


    # In[22]:


    count = 0
    last_index = (rows_in_each_table['Profit & Loss']*columns_in_each_table['Profit & Loss'])-1
    for i in range(rows_in_each_table['Profit & Loss']):
        temp_row = []
        for j in range(columns_in_each_table['Profit & Loss']):
            if(count == last_index):
                temp_row.append(np.nan)
                break
            temp_element = table2_data_only[count]
            temp_element = temp_element.replace(',','')
            temp_element = temp_element.replace('%','')
            
            if temp_element == 'empty':
                temp_element = np.nan
            
            temp_row.append(float(temp_element))
            count = count + 1
        df_Profit_and_Loss.loc[len(df_Profit_and_Loss)] = temp_row


    # In[23]:


    #now renaming index for table2
    df_Profit_and_Loss = df_Profit_and_Loss.rename(index={0: 'Sales', 1: 'Expenses',2: 'Operating Profit', 3: 'OPM %',4: 'Other Income', 5: 'Interest',6: 'Depreciation', 7: 'Profit before tax',8: 'Tax %', 9: 'Net Profit', 10: 'EPS in Rs', 11: 'Dividend Payout %'})


    # In[24]:


    table3_data_only = []
    for t in table3:
        if t[0].isdigit() == False and (not (t[0] == "-") and (not (t == "empty"))):
            continue
        else:
            table3_data_only.append(t)


    # In[25]:


    count = 0
    for i in range(rows_in_each_table['Balance Sheet']):
        temp_row = []
        for j in range(columns_in_each_table['Balance Sheet']):
            temp_element = table3_data_only[count]
            temp_element = temp_element.replace(',','')
            temp_element = temp_element.replace('%','')
            
            if temp_element == 'empty':
                temp_element = np.nan
            
            temp_row.append(float(temp_element))
            count = count + 1
        df_Balance_Sheet.loc[len(df_Balance_Sheet)] = temp_row


    # In[26]:


    #now renaming index for table3
    df_Balance_Sheet = df_Balance_Sheet.rename(index={0: 'Share Capital', 1: 'Reserves',2: 'Borrowings', 3: 'Other Liabilities',4: 'Total Liabilities', 5: 'Fixed Assets',6: 'CWIP', 7: 'Investments',8: 'Other Assets', 9: 'Total Assets'})


    # In[27]:


    table4_data_only = []
    for t in table4:
        if t[0].isdigit() == False and (not (t[0] == "-") and (not (t == "empty"))):
            continue
        else:
            table4_data_only.append(t)


    # In[28]:


    count = 0
    for i in range(rows_in_each_table['Cash Flow']):
        temp_row = []
        for j in range(columns_in_each_table['Cash Flow']):
            temp_element = table4_data_only[count]
            temp_element = temp_element.replace(',','')
            temp_element = temp_element.replace('%','')
            
            if temp_element == 'empty':
                temp_element = np.nan
            
            temp_row.append(float(temp_element))
            count = count + 1
        df_Cash_Flow.loc[len(df_Cash_Flow)] = temp_row


    # In[29]:


    #now renaming index for table4
    df_Cash_Flow = df_Cash_Flow.rename(index={0: 'Cash from Operating Activity', 1: 'Cash from Investing Activity',2: 'Cash from Financing Activity', 3: 'Net Cash Flow'})


    # In[30]:


    #now table 5
    table5_data_only = []
    for t in table5:
        if t[0].isdigit() == False and (not (t[0] == "-") and (not (t == "empty"))):
            continue
        else:
            table5_data_only.append(t)


    # In[31]:


    count = 0
    for i in range(rows_in_each_table['Ratios']):
        temp_row = []
        for j in range(columns_in_each_table['Ratios']):
            temp_element = table5_data_only[count]
            temp_element = temp_element.replace(',','')
            temp_element = temp_element.replace('%','')
            
            if temp_element == 'empty':
                temp_element = np.nan
            
            temp_row.append(float(temp_element))
            count = count + 1
        df_Ratios.loc[len(df_Ratios)] = temp_row


    # In[32]:


    #now renaming index for table5
    df_Ratios = df_Ratios.rename(index={0: 'ROCE %', 1: 'Debtor Days',2: 'Inventory Turnover'})


    # In[33]:


    dataset = {'Main_Info':df_main_info,'Quarters':df_Quarters, 'Profit_and_Loss':df_Profit_and_Loss, 'Balance_Sheet':df_Balance_Sheet, 'Cash_Flow':df_Cash_Flow, 'Ratios':df_Ratios, 'Compounded_Sales_Growth':df_Compounded_Sales_Growth , 'Compounded_Profit_Growth':df_Compounded_Profit_Growth , 'Stock_Price_CAGR':df_Stock_Price_CAGR , 'Return_on_Equity':df_Return_on_Equity }


    # In[34]:


    # Create a Pandas Excel writer using XlsxWriter as the engine.
    save_path = 'datastore/'
    complete_name = name_number + c_name + '.xlsx'
    writer = pd.ExcelWriter(save_path+complete_name, engine='xlsxwriter')

    # Write each dataframe to a different worksheet.
    dataset['Main_Info'].to_excel(writer, sheet_name='Main_Info')
    dataset['Quarters'].to_excel(writer, sheet_name='Quarters')
    dataset['Profit_and_Loss'].to_excel(writer, sheet_name='Profit_and_Loss')
    dataset['Balance_Sheet'].to_excel(writer, sheet_name='Balance_Sheet')
    dataset['Cash_Flow'].to_excel(writer, sheet_name='Cash_Flow')
    dataset['Ratios'].to_excel(writer, sheet_name='Ratios')

    dataset['Compounded_Sales_Growth'].to_excel(writer, sheet_name='Compounded_Sales_Growth')
    dataset['Compounded_Profit_Growth'].to_excel(writer, sheet_name='Compounded_Profit_Growth')
    dataset['Stock_Price_CAGR'].to_excel(writer, sheet_name='Stock_Price_CAGR')
    dataset['Return_on_Equity'].to_excel(writer, sheet_name='Return_on_Equity')

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

https://www.screener.in/company/EICHERMOT/consolidated/
https://www.screener.in/company/BAJAJ-AUTO/consolidated/
